In [1]:
!git clone https://github.com/ksanjeevan/crnn-audio-classification.git

Cloning into 'crnn-audio-classification'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 127 (delta 63), reused 101 (delta 37), pack-reused 0
Receiving objects: 100% (127/127), 3.55 MiB | 0 bytes/s, done.
Resolving deltas: 100% (63/63), done.


In [2]:
!git clone https://github.com/keunwoochoi/torchaudio-contrib
!pip install -e torchaudio-contrib

Cloning into 'torchaudio-contrib'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 209 (delta 22), reused 34 (delta 11), pack-reused 155
Receiving objects: 100% (209/209), 1.89 MiB | 0 bytes/s, done.
Resolving deltas: 100% (99/99), done.
Obtaining file:///home/jupyter/crnn-audio-classification%20-%20UrbanSound8k/torchaudio-contrib
  Running setup.py develop for torchaudio-contrib


In [3]:
!cp -r crnn-audio-classification/* .

In [13]:
!pip install SoundFile
!pip install git+https://github.com/ksanjeevan/torchparse.git
!pip install tensorboardX
!pip install tensorboard
!pip install tensorflow

  Cloning https://github.com/ksanjeevan/torchparse.git to /tmp/pip-req-build-cxyj2nui
  Running setup.py bdist_wheel for torchparse ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-pndy0yu8/wheels/53/64/f2/c60bf851fcf5d5363538889a115dd68d728474bbc22ef7d280
Successfully built torchparse


In [ ]:
!wget https://goo.gl/8hY5ER
!tar -zxvf 8hY5ER
!rm -f 8hY5ER

In [2]:
json_config = {
    "name"          :   "Urban Testing",
    "data"          :   {
                            "type"      :   "CSVDataManager",
                            "path"      :   "UrbanSound8K",
                            "format"    :   "audio",
                            "loader"    :   {
                                                "shuffle"       : True,
                                                "batch_size"    : 24,
                                                "num_workers"   : 4,
                                                "drop_last"     : True
                                            },
                            "splits"    :   {
                                                "train" : [1,2,3,4,5,6,7,8,9], 
                                                "val"   : [10]                                            
                                            }
                        },
    "transforms"    :   {
                            "type"      :   "AudioTransforms",
                            "args"      :   {
                                                "channels"       : "avg",
                                                "noise"    : [0.3, 0.001],
                                                "crop"     : [0.4, 0.25]
                                            }
                        },
    "optimizer"     :   {
                            "type"      :   "Adam",
                            "args"      :   {
                                                "lr"            : 0.002,
                                                "weight_decay"  : 0.01,
                                                "amsgrad"       : True
                                            }
                        },
    "lr_scheduler"   :   {
                            "type"      :   "StepLR",
                            "args"      :   {
                                                "step_size" : 10,
                                                "gamma"     : 0.5
                                            }
                        },
    "model"         :   {
                            "type"      :   "AudioCRNN"
                        },
    "train"         :   {
                            "loss"      :   "nll_loss",
                            "epochs"    :   10,
                            "save_dir"  :   "saved_cv/",
                            "save_p"    :   1,
                            "verbosity" :   2,
                            
                            "monitor"   :   "min val_loss",
                            "early_stop":   8,
                            "tbX"       :   True
                        },
    "metrics"       :   "classification_metrics"

}

In [3]:
import json
with open('my-config.json', 'w') as json_file:  
  json.dump(json_config, json_file)

In [4]:
!python run.py train -c my-config.json --cfg crnn.cfg

Compose(
    ProcessChannels(mode=avg)
    AdditiveNoise(prob=0.3, sig=0.001, dist_type=normal)
    RandomCropLength(prob=0.4, sig=0.25, dist_type=half)
    ToTensorAudio()
)
AudioCRNN(
  (spec): MelspectrogramStretch(num_bands=128, fft_len=2048, norm=spec_whiten, stretch_param=[0.4, 0.4])
  (net): ModuleDict(
    (convs): Sequential(
      (conv2d_0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=[0, 0])
      (batchnorm2d_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (elu_0): ELU(alpha=1.0)
      (maxpool2d_0): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
      (dropout_0): Dropout(p=0.1)
      (conv2d_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=[0, 0])
      (batchnorm2d_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (elu_1): ELU(alpha=1.0)
      (maxpool2d_1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    

In [15]:
!python run.py UrbanSound8K/audio/fold10/100795-3-0-0.wav -r saved_cv/0515_171217/checkpoints/model_best.pth

dog_bark 0.9858338236808777


In [10]:
!tensorboard --logdir=saved_cv/0515_171217/logs

TensorBoard 1.13.1 at http://my-fastai-instance:6006 (Press CTRL+C to quit)
^C


In [9]:
!pip install tensorboard
!pip install tensorflow


    100% |████████████████████████████████| 92.6MB 356kB/s 
    100% |████████████████████████████████| 61kB 33.4MB/s 
    100% |████████████████████████████████| 61kB 35.8MB/s 
    100% |████████████████████████████████| 368kB 43.6MB/s 
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built termcolor gast
